In [1]:
import pandas as pd
from docplex.mp.model import Model
from docplex.mp.linear import LinearExpr

In [16]:
#Leitura das Instâncias
veiculos = pd.read_csv("k_15",index_col=0)
vitimas = pd.read_csv("v_5",index_col=0)
pontos = pd.read_csv("n_150",index_col=0)
hospitais = veiculos["Inicial"].unique()

In [17]:
#Criação da Matriz de Distância
_DistanceMatrix = pd.DataFrame(0,index=pontos.index,columns=pontos.index,dtype=float)

In [18]:
#Popula a Matriz de Distância
for i in _DistanceMatrix.index:
    for j in pontos.loc[i][2:].values:
        x = (pontos.loc[i]["x"]-pontos.loc[int(j)]["x"])**2
        y = (pontos.loc[i]["y"]-pontos.loc[int(j)]["y"])**2
        _DistanceMatrix.loc[i][j] = 100*((x+y)**(1/2))

In [5]:
vitimas

# Modelagem

In [6]:
vrp = Model("Ambulancias-VRP") #Modelo

In [7]:
#Matriz com as variáveis de decisão principais
_DecisionCube = vrp.binary_var_cube(_DistanceMatrix.index,_DistanceMatrix.index,veiculos.index,"Arc")

In [8]:
#Função Objetivo
vrp.minimize(vrp.sum(_DecisionCube[i,j,k]*_DistanceMatrix.loc[i][j] 
                     for i in _DistanceMatrix.index 
                     for j in _DistanceMatrix.index 
                     for k in veiculos.index))

In [9]:
forbidden = LinearExpr(vrp)   #Arcos proíbidos

for k in veiculos.index:
    for i in _DistanceMatrix.index:
        
        flux = LinearExpr(vrp)    #Conservação de Fluxo
        match = LinearExpr(vrp)   #Restrição de Compatibilidade
        for j in _DistanceMatrix.index:
            
            if _DistanceMatrix.loc[i][j] > 0:    #Verifica se o arco existe
                
                if i in vitimas["Ponto"].values: #Se nodo 'i' for uma vítima:
                    match+=_DecisionCube[j,i,k]  #Adiciona os arcos 'jik' em 'match'
                    flux +=_DecisionCube[j,i,k]  #Subtrai o arco de chegada em 'i'
                
                elif i == veiculos.loc[k]["Inicial"]: #Se o nodo 'i' for um hospital
                    flux += _DecisionCube[i,j,k]      #Adiciona o arco de saída de 'i'
                
                elif j == veiculos.loc[k]["Inicial"]: #Se o nodo 'j' for o ponto de saida de 'k'
                    forbidden += _DecisionCube[i,j,k] #Retira o arco 'ij' da solução
                    flux -=_DecisionCube[j,i,k]       #Coloca o arco de saída de 'j' no fluxo
                    
                else:                                 #Se nenhum desses casos se aplicar
                    flux+=_DecisionCube[i,j,k]-_DecisionCube[j,i,k] #Fluxo normal
                
                    
            else:                               #Se o arco não existir
                forbidden += _DecisionCube[i,j,k]#Tira ele da solução
                         
        if i in vitimas["Ponto"].values: #Se nodo 'i' for uma vítima
            #Permite 'k' chegar em 'i' no máximo 1 vez
            vrp.add_constraint(flux<=1,"ChegadaVitima_"+str(i)+"_Carro_"+str(k))
            
            #Pega a prioridade da vitima 'i'
            prioridade = vitimas[vitimas["Ponto"] == i]["Prioridade"].values[0]
            
            #E cria as restriçõe de UpperBound
            vrp.add_constraint((veiculos.loc[k]["UpperBound"])*match-prioridade*match>=0,
                               "MatchUB_Ocurrence_"+str(i)+"_Vehicle"+str(k))
            #E LowerBound
            vrp.add_constraint((veiculos.loc[k]["LowerBound"])*match-prioridade*match<=0,
                              "MatchLB_Ocurrence_"+str(i)+"_Vehicle"+str(k))
       
        elif i == veiculos.loc[k]["Inicial"]: #Se o nodo 'i' for o inicial do veiculo 'k'
            #Permite 'k' sair de 'i' no máximo 1 vez
            vrp.add_constraint(flux<=1,"FluxoInicialVeiculo_"+str(k))
        else:
            #Fluxo Normal
            vrp.add_constraint(flux==0,"Fluxo_"+str(i)+"_Carro_"+str(k))

#Eliminação dos arcos inválidos
vrp.add_constraint(forbidden==0,"Cuts");

In [10]:
for k in veiculos.index:
    
    inicial = veiculos.loc[k]["Inicial"] #Ponto de Saída de 'k'
    
    flux_inicial = LinearExpr(vrp) #Fluxo de Saída de 'k'
    
    for i in pontos.loc[inicial][2:].values:          #Para todos os arcos saindo do inicial
        flux_inicial+=_DecisionCube[inicial,int(i),k] #Adicione-os ao fluxo de saída
    
    flux_final = LinearExpr(vrp) #Fluxo de Chegada nas ocorrências
    
    for i in vitimas["Ponto"].values:          #Para todas as ocorrências
        for j in pontos.loc[i][2:]:            #Para todos os arcos saindo delas
            flux_final += _DecisionCube[j,i,k] #Adicione-os ao fluxo de chegada
            
    #Se o veículo sair de seu ponto inicial, ele deve atender uma ocorrência
    vrp.add_constraint(flux_inicial-flux_final==0,"AtendimentoObrigatorio_"+str(k))

In [11]:
for i in vitimas["Ponto"].values:
    
    demand = LinearExpr(vrp) #Demanda da vítima 'i'
    
    for j in pontos.loc[i][2:].values:
        for k in veiculos.index:
            #Para cada arco 'j' chegando em 'i' e veículo 'k'
            #Os adicionamos à expressão
            demand+=_DecisionCube[int(j),int(i),k]
    #Ao menos um precisa passar nessa vítima
    vrp.add_constraint(demand>=1,"DemandaVitima_"+str(i))

In [12]:
for k in veiculos.index:
    capacidade = LinearExpr(vrp) #Capacidade do veículo 'k'
    for i in vitimas["Ponto"].values:
        for j in pontos.loc[i][2:]:
            #Para cada arco chegando em cada vítima
            #Adicione-o à expressão
            capacidade+=_DecisionCube[int(j),int(i),k]
            
    #O veículo só pode 
    vrp.add_constraint(capacidade<=1,"CapacidadeVeiculo_"+str(k))

In [13]:
vrp.export_as_lp("Model")

'Model.lp'

In [14]:
import time
start = time.time()
subcicle = True
#Assumindo que existe subciclos, nós...
while subcicle == True:
    #... criamos uma rota para cada entregador...
    routes = {x:{} for x in veiculos.index}    
    #... e solucionamos o vrpo.
    _Solution = vrp.solve(log_output=True).as_dict()
    #Separamos a solução em rotas distintas para cada entregador...
    for i in _Solution:
        aux = i.name.split("_")[1:]
        routes[int(aux[2])][int(aux[0])]=int(aux[1])
    
    count = 0
    summ = 0
    #... percorremos toda a lista de de entregadores...
    for k in veiculos.index:
        #... verificando se ele fez alguma rota...
        if len(routes[k]) > 0:
            #... e caso tenha feito, retiramos a rota principal.
            inicial = veiculos.loc[k]["Inicial"]
            if inicial in list(routes[k].keys()):
                actual_node = routes[k].pop(inicial)
            else:
                inicial =list(routes[k].keys())[0]
                actual_node = routes[k].pop(inicial)
            while actual_node not in vitimas["Ponto"].values:
                actual_node = routes[k].pop(actual_node) #Percorre o ciclo principal e o exclui#
        summ += len(routes[k])
        
        
        #Verificamos em seguida se sobrou algo depois de retirarmos a rota principal...
        if len(routes[k]) >0:
            #... e percorremos todas os subciclos ...
            while(len(routes[k])>0):
                cut = LinearExpr(vrp)
                first_node = list(routes[k].keys())[0]
                actual_node = routes[k].pop(first_node)
                cut += _DecisionCube[int(first_node),int(actual_node),k]
                size = 0
                #... retirando-os 1 de cada vez ...
                
                while(actual_node != first_node):
                    next_node = routes[k].pop(actual_node)                 #Percorre o subciclo e o adiciona ao#
                    cut += _DecisionCube[int(actual_node),int(next_node),k]#conjunto de subciclos proíbidos.#
                    actual_node = next_node
                    size+=1
                count+=1
                vrp.add_constraint(cut<=size,"SubCycleCut_"+str(count))
    if summ == 0:
        subcicle = False
print(time.time()-start)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 1 time.
MIP Presolve eliminated 219 rows and 173016 columns.
MIP Presolve modified 8 coefficients.
Reduced MIP has 1766 rows, 6984 columns, and 14028 nonzeros.
Reduced MIP has 6984 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.23 sec. (72.92 ticks)
Found incumbent of value 26849.696638 after 0.56 sec. (120.40 ticks)
Probing time = 0.10 sec. (0.58 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 97 rows and 384 columns.
Reduced MIP has 1669 rows, 6600 columns, and 13276 nonzeros.
Reduced MIP has 6600 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (9.58 ticks)
Probing time = 0.07 sec. (0.55 ticks)
Clique table members: 34.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic

In [15]:
#Cria novamente as rotas de cada veículo
for i in _Solution:
        aux = i.name.split("_")[1:]
        routes[int(aux[2])][int(aux[0])]=int(aux[1])
for k in veiculos.index:
    if len(routes[k]) > 0:
        inicial = veiculos.loc[k]["Inicial"]
        actual_node = routes[k].pop(inicial)
        
        string=str(k)+": "+str(inicial)+" -> "+str(actual_node)
        while actual_node not in vitimas["Ponto"].values:
            actual_node = routes[k].pop(actual_node)
            string+="-> "+str(actual_node)
            
        print(string)

1: 71 -> 91-> 9
8: 41 -> 4-> 49
15: 47 -> 45-> 38-> 90-> 17-> 39-> 77-> 19-> 22-> 16-> 61-> 48-> 59
16: 47 -> 45-> 38-> 90-> 17-> 39-> 77-> 5-> 85
